In [1]:
## Imports

## (Fractal) Images of the Mandelbrot set

In [ ]:
## Add code for Images

## Convergence of no. iterations

In [ ]:
## Add plot

## Sampling methods

In [ ]:
## Show sampling methods

## Convergence of the Area

In [ ]:
## Add plots of convergence 

## Improving the convergence